**Human Language Technologies Project**

**Authors:** Dalla Noce Niko, Ristori Alessandro

# **HLT Project**

This work is higly based on the tensorflow tutorial https://www.tensorflow.org/text/tutorials/transformer, our aim was to introduce BERT as an encoder in the model and try combinations with different architectures (both RNNs and transformers).

## **Setup**
We need to install the transformers package to use the models and tokenizers from HuggingFace.

In [13]:
!pip install transformers
!pip install sentencepiece

In [14]:
import logging
import sentencepiece
import random
import numpy as np
from tqdm.notebook import tqdm_notebook
import tensorflow as tf
from tensorflow.keras import layers

The model training is going to run on TPUs since they are the optimized for working with tensors, if there are no TPUs avilable then we work with a GPU instead.

In [15]:
# Detect hardware
try:
    tpu_resolver = tf.distribute.cluster_resolver.TPUClusterResolver() # TPU detection
except ValueError:
    tpu_resolver = None
    gpus = tf.config.experimental.list_logical_devices("GPU")

# Select appropriate distribution strategy
if tpu_resolver:
    tf.config.experimental_connect_to_cluster(tpu_resolver)
    tf.tpu.experimental.initialize_tpu_system(tpu_resolver)
    strategy = tf.distribute.experimental.TPUStrategy(tpu_resolver)
    print('Running on TPU ', tpu_resolver.cluster_spec().as_dict()['worker'])
elif len(gpus) > 1:
    strategy = tf.distribute.MirroredStrategy([gpu.name for gpu in gpus])
    print('Running on multiple GPUs ', [gpu.name for gpu in gpus])
elif len(gpus) == 1:
    strategy = tf.distribute.get_strategy() # default strategy that works on CPU and single GPU
    print('Running on single GPU ', gpus[0].name)
else:
    strategy = tf.distribute.get_strategy() # default strategy that works on CPU and single GPU
    print('Running on CPU')
  
print("Number of accelerators: ", strategy.num_replicas_in_sync)

Running on single GPU  /device:GPU:0
Number of accelerators:  1


In [16]:
logging.getLogger('tensorflow').setLevel(logging.ERROR)  # suppress warnings

Clone the project repository.

In [17]:
!git clone "https://github.com/nikodallanoce/HLT/"  # clone the project repository

fatal: destination path 'HLT' already exists and is not an empty directory.


Mount the drive.

In [18]:
from google.colab import drive
drive.mount('/content/drive')  # mount the drive

Mounted at /content/drive


Import all the models needed for the project.

In [19]:
def create_encoder_bert():
    with strategy.scope():
        bert_model = TFBertModel.from_pretrained("bert-base-cased")
    return bert_model

def create_encoder_t5():
    with strategy.scope():
        t5_model = TFT5EncoderModel.from_pretrained("t5-base"),
    return t5_model

def create_encoder_distilbert():
    with strategy.scope():
        distilbert_model = TFDistilBertModel.from_pretrained("distilbert-base-cased"),
    return distilbert_model

def create_encoder_xlnet():
    with strategy.scope():
        xlnet_model = TFXLNetModel.from_pretrained("xlnet-base-cased"),
    return xlnet_model

In [20]:
from transformers import BertTokenizer, BertTokenizerFast, TFBertModel, T5TokenizerFast, T5Tokenizer, TFT5EncoderModel, DistilBertTokenizer, DistilBertTokenizerFast, TFDistilBertModel, XLNetTokenizerFast, XLNetTokenizer, TFXLNetModel

logging.getLogger("transformers").setLevel(logging.ERROR)  # suppress warning for transformers

encoder_models = {
    "bert": {
        "tokenizer": BertTokenizerFast.from_pretrained("bert-base-cased"),
        "encoder": create_encoder_bert,
        "tokenizer_translation": BertTokenizer.from_pretrained("bert-base-cased"),
    },
    "t5": {
        "tokenizer": T5TokenizerFast.from_pretrained("t5-base"),
        "encoder" : create_encoder_t5,
        "tokenizer_translation" : T5Tokenizer.from_pretrained("t5-base"),
    },
    "distil_bert" : {
        "tokenizer" : DistilBertTokenizerFast.from_pretrained("distilbert-base-cased"),
        "encoder" : create_encoder_distilbert,
        "tokenizer_translation" : DistilBertTokenizer.from_pretrained("distilbert-base-cased"),
    },
    "xlnet" : {
        "tokenizer" : XLNetTokenizerFast.from_pretrained("xlnet-base-cased"),
        "encoder" : create_encoder_xlnet,
        "tokenizer_translation" : XLNetTokenizer.from_pretrained("xlnet-base-cased"),
    },
}

Downloading:   0%|          | 0.00/208k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/426k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/29.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/773k [00:00<?, ?B/s]

...

Downloading:   0%|          | 0.00/29.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/411 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/779k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.32M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/760 [00:00<?, ?B/s]

In [21]:
from transformers import AlbertTokenizerFast, AlbertTokenizer, TFAlbertModel

albert_tokenizer = AlbertTokenizerFast.from_pretrained("albert-base-v2")
albert_model = TFAlbertModel.from_pretrained("albert-base-v2")

Downloading:   0%|          | 0.00/742k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.25M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/684 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/60.1M [00:00<?, ?B/s]

## **Building the training, validation and test set**

We need to build our three sets by preprocessing the en-it anki dataset, tokenizing it and splitting it into training, validation and test.

### **Preprocessing the dataset**

Create two lists containing the sentences of the anki dataset, one in english and one in italian.

Let's define the method to preprocess the anki dataset.

In [ ]:
def create_dataset_euparl(name: str, src: str = "en", dst: str = "it", size: float = 1) -> (list, list):
    with open(name+".{0}".format(src), encoding="UTF-8") as datafile:
        src_set = datafile.readlines()

    with open(name+".{0}".format(dst), encoding="UTF-8") as datafile:
        dst_set = datafile.readlines()

    if size != 1:
        datasets_to_shuffle = list((zip(src_set, dst_set)))
        np.random.shuffle(datasets_to_shuffle)
        src_set, dst_set = zip(*datasets_to_shuffle)
        src_set = list(src_set[:int(len(src_set) * size)])
        dst_set = list(dst_set[:int(len(dst_set) * size)])

    return src_set, dst_set

def create_dataset_anki(name: str, preprocessed:bool) -> (list, list):
    with open(name, encoding="UTF-8") as datafile:
        src_set = list()
        dst_set = list()
        for sentence in datafile:
            sentence = sentence.split("\t")
            src_set.append(sentence[0])
            if preprocessed:
                dst_set.append(sentence[1].split("\n")[0])
            else:
                dst_set.append(sentence[1])

    return src_set, dst_set

The dataset is in the project repository inside a zip file, we need to extract it and then we can build our lists using the previous method.

In [ ]:
import tarfile
fname = "/content/drive/Shareddrives/HLT/datasets/it-en.tar"
if fname.endswith("tar.gz"):
    tar = tarfile.open(fname, "r:gz")
    tar.extractall()
    tar.close()
elif fname.endswith("tar"):
    tar = tarfile.open(fname, "r:")
    tar.extractall()
    tar.close()

In [ ]:
import zipfile
with zipfile.ZipFile("HLT/dataset/dataset_anki_it.zip", 'r') as zip_ref:
    zip_ref.extractall("")

# en_set, it_set = create_dataset_anki("ita_preprocessed.txt", True)
en_set, it_set = create_dataset_euparl("europarl-v7.it-en", size=0.5)
print("The corpus' size is: {0}".format(len(en_set)))

The corpus' size is: 954557


### **Dataset tokenization**

We tokenize each sentence in the two lists by using the tokenizers from huggingface.

Before we create the dataset from our lists, we have to tokenize each sentence from the corpus by using the BERT tokenizer for english and the one for italian. Moreover we can get the number of tokens for both source and target.

In [ ]:
encoder_model = encoder_models["bert"]
ita_src = "dbmdz/bert-base-italian-cased"

In [ ]:
# Create the tokenizers and get the number of tokens
tokenizer_en = encoder_model["tokenizer"]
tokenizer_it = BertTokenizerFast.from_pretrained(ita_src)
v_size_en = tokenizer_en.vocab_size
v_size_it = tokenizer_it.vocab_size

print("Number of tokens for the english dataset: {0}".format(v_size_en))
print("Number of tokens for the italian dataset: {0}".format(v_size_it))

Downloading:   0%|          | 0.00/230k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/59.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/433 [00:00<?, ?B/s]

Number of tokens for the english dataset: 28996
Number of tokens for the italian dataset: 31102


Let's calculate the max number of tokens allowed, this number is taken such that 99% of the sentences in the dataset are fully tokenized.

In [ ]:
def set_max_tokens(dataset: list, language: str = "en") -> int:
    len_sentences = [len(sentence.split()) for sentence in dataset]
    mean_len_sentences = np.mean(len_sentences)
    print("{0} dataset average sentence length: {1}".format(language, mean_len_sentences))
    max_length = int(mean_len_sentences + 2 * np.std(len_sentences))
    print("{0} dataset max length allowed: {1}".format(language, max_length))
    return max_length

max_length_en = set_max_tokens(en_set, "en")
max_length_it = set_max_tokens(it_set, "it")

en dataset average sentence length: 26.05064338745617
en dataset max length allowed: 56
it dataset average sentence length: 25.137066723097732
it dataset max length allowed: 55


Tokenize the source and target dataset.

In [ ]:
# Tokenize the dataset
max_length = np.max([max_length_en, max_length_it])  # use just one of the max length allowed
with strategy.scope():
    tokens_en = tokenizer_en(en_set, add_special_tokens=True,
                              truncation=True, padding="max_length", return_attention_mask=True,
                              return_tensors="tf", max_length=max_length).data["input_ids"]
    tokens_it = tokenizer_it(it_set, add_special_tokens=True,
                              truncation=True, padding="max_length", return_attention_mask=True,
                              return_tensors="tf", max_length=max_length+1).data["input_ids"]

KeyboardInterrupt: ignored

Let's show some sentences from both languages and how they were tokenized.

In [ ]:
for _ in range(3):
  i = np.random.randint(len(tokens_en))
  print("En sentence: {0}\nTokenized sentence: {1}".format(en_set[i], tokens_en[i]))
  print("It sentence: {0}\nTokenized sentence: {1}\n".format(it_set[i], tokens_it[i]))

En sentence: After all, the previous directive dates from 1996 - that is, 14 years ago!

Tokenized sentence: [ 101 1258 1155  117 1103 2166 2904 2109 4595 1121 1820  118 1115 1110
  117 1489 1201 2403  106  102    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0]
It sentence: Dopo tutto, la direttiva precedente risale addirittura al 1996, vale a dire 14 anni fa!

Tokenized sentence: [ 102 1566  564 1307  146 1724 2865 9678 7431  157 6430 1307 4426  111
  499 1285  578  231 2692  103    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0]

En sentence: I believe that both of these approaches are wrong.

Tokenized sentence: [ 101  146 2059 1115 1241 1104 1292 8015 1132 2488  119  102    0    0
    0    0    0    0    0    0    0 

### **Splitting the dataset**

Then we build the tf dataset and split it into training, validation and test sets.

In [ ]:
def split_set(dataset: tf.data.Dataset,
              tr: float = 0.8,
              val: float = 0.1,
              ts: float = 0.1,
              shuffle: bool = True) -> (tf.data.Dataset, tf.data.Dataset, tf.data.Dataset):
    if tr+val+ts != 1:
        raise ValueError("Train, validation and test partition not allowed with such splits")

    dataset_size = dataset.cardinality().numpy()
    if shuffle:
        dataset = dataset.shuffle(dataset_size)

    tr_size = int(tr * dataset_size)
    val_size = int(val * dataset_size)

    tr_set = dataset.take(tr_size)
    val_set = dataset.skip(tr_size).take(val_size)
    ts_set = dataset.skip(tr_size).skip(val_size)
    return tr_set, val_set, ts_set

In [ ]:
# Build the dataset and split it in train, validation and test
dataset = tf.data.Dataset.from_tensor_slices((tokens_en, tokens_it))  # build the tf dataset
tr_set, val_set, ts_set = split_set(dataset, 0.8, 0.1, 0.1)  # split the tf dataset
print("Training set size: {0}".format(len(tr_set)))
print("Validation set size: {0}".format(len(val_set)))
print("Test set size: {0}".format(len(ts_set)))

Training set size: 763645
Validation set size: 95455
Test set size: 95457


### **Create training and validation batches**

After we have built our development and test set, we need to split the first one (both training and validation) in batches.


In [ ]:
def format_dataset(eng, ita):
    return ({"encoder_inputs": eng, "decoder_inputs": ita[:, :-1],}, ita[:, 1:])

def make_batches(dataset_src_dst: tf.data.Dataset, batch_size: int) -> tf.data.Dataset:
    dataset = dataset_src_dst.batch(batch_size)
    dataset = dataset.map(format_dataset)
    return dataset.prefetch(tf.data.experimental.AUTOTUNE).cache()

In [ ]:
batch_size =  16 * strategy.num_replicas_in_sync

with strategy.scope():
    tr_batches = make_batches(tr_set, batch_size)
    val_batches = make_batches(val_set, batch_size)

NameError: ignored

In [ ]:
for src, dst in tr_batches.take(1):
    print("encoder inputs shape: {0}".format(src["encoder_inputs"].shape))
    print("decoder inputs shape: {0}".format(src["decoder_inputs"].shape))
    print("targets shape: {0}".format(dst.shape))

encoder inputs shape: (128, 56)
decoder inputs shape: (128, 56)
targets shape: (128, 56)


## **Layers**

Our sequence-to-sequence Transformer consists of a `TransformerEncoder`
and a `TransformerDecoder` chained together.

### **Positional embeddings layer**

To make the model aware of word order, we also use a PositionalEmbedding layer.

In [ ]:
class PositionalEmbedding(layers.Layer):
    def __init__(self, sequence_length, v_size, embed_dim, **kwargs):
        super(PositionalEmbedding, self).__init__(**kwargs)
        self.token_embeddings = layers.Embedding(
            input_dim=v_size, output_dim=embed_dim
        )
        self.position_embeddings = layers.Embedding(
            input_dim=sequence_length, output_dim=embed_dim
        )
        self.sequence_length = sequence_length
        self.v_size = v_size
        self.embed_dim = embed_dim

    def call(self, inputs):
        length = tf.shape(inputs)[-1]
        positions = tf.range(start=0, limit=length, delta=1)
        embedded_tokens = self.token_embeddings(inputs)
        embedded_positions = self.position_embeddings(positions)
        return embedded_tokens + embedded_positions

    def compute_mask(self, inputs, mask=None):
        return tf.math.not_equal(inputs, 0)

### **Encoder**

The source sequence will be pass to the TransformerEncoder, which will produce a new representation of it. This new representation will then be passed to the TransformerDecoder, together with the target sequence so far (target words 0 to N). 

The single layer of the encoder transformer.

In [ ]:
class EncoderLayer(layers.Layer):

    def __init__(self, layers_size: int, dense_size: int, num_heads: int, dropout=0.1, **kwargs) -> None:
        super(EncoderLayer, self).__init__(**kwargs)
        
        self.layers_size = layers_size
        self.dense_size = dense_size
        self.num_heads = num_heads
        self.attention = layers.MultiHeadAttention(num_heads, layers_size)
        self.dense_proj = tf.keras.Sequential(
            [layers.Dense(dense_size, activation="relu"), layers.Dense(layers_size)]
        )
        self.layernorm_1 = layers.LayerNormalization()
        self.layernorm_2 = layers.LayerNormalization()
        self.dropout_1 = layers.Dropout(dropout)
        self.dropout_2 = layers.Dropout(dropout)
        self.supports_masking = True

    def call(self, inputs: tf.Tensor, mask=None) -> tf.Tensor:
        if mask is not None:  
            padding_mask = tf.cast(mask[:, tf.newaxis, tf.newaxis, :], dtype="int32")
        else:
            print("Mask not built")
            assert False
        
        attention_output = self.attention(
            query=inputs, value=inputs, key=inputs, attention_mask=padding_mask
        )
        attention_output = self.dropout_1(attention_output)
        proj_input = self.layernorm_1(inputs + attention_output)
        proj_output = self.dense_proj(proj_input)
        proj_output = self.dropout_1(proj_output)
        return self.layernorm_2(proj_input + proj_output)

The encoder transformer.

In [ ]:
class EncoderTransformer(layers.Layer):

    def __init__(self,
                 num_layers: int,
                 layers_size: int,
                 dense_size: int,
                 num_heads: int,
                 max_length: int,
                 v_size_src: int,
                 dropout: float = 0.1) -> None:
        super(EncoderTransformer, self).__init__()

        self.layers_size = layers_size
        self.num_layers = num_layers
        self.pos_embedding = PositionalEmbedding(max_length, v_size_src, layers_size)
        self.enc_layers = [EncoderLayer(layers_size, dense_size, num_heads) for _ in range(num_layers)]
        self.dropout = tf.keras.layers.Dropout(dropout)
        self.supports_masking = True

    def call(self, inputs: tf.Tensor, mask=None) -> tf.Tensor:
        src_embeddings = self.pos_embedding(inputs)
        enc_out = self.dropout(src_embeddings)
        for i in range(self.num_layers):
            enc_out = self.enc_layers[i](enc_out)

        return enc_out  # (batch_size, input_seq_len, layers_size)

### **Decoder**

The `TransformerDecoder` will then seek to predict the next words in the target sequence (N+1 and beyond).

The single layer of the decoder transformer.

In [ ]:
class DecoderLayer(layers.Layer):

    def __init__(self, layers_size: int, dense_size: int, num_heads: int, dropout=0.1, **kwargs) -> None:
        super(DecoderLayer, self).__init__(**kwargs)
        
        self.layers_size = layers_size
        self.dense_size = dense_size
        self.num_heads = num_heads
        self.attention_1 = layers.MultiHeadAttention(num_heads, layers_size)
        self.attention_2 = layers.MultiHeadAttention(num_heads, layers_size)
        self.dense_proj = tf.keras.Sequential(
            [layers.Dense(dense_size, activation="relu"), layers.Dense(layers_size)]
        )
        self.layernorm_1 = layers.LayerNormalization()
        self.layernorm_2 = layers.LayerNormalization()
        self.layernorm_3 = layers.LayerNormalization()
        self.dropout_1 = layers.Dropout(dropout)
        self.dropout_2 = layers.Dropout(dropout)
        self.dropout_3 = layers.Dropout(dropout)
        self.supports_masking = True

    def call(self, inputs: tf.Tensor, encoder_outputs: tf.Tensor, mask=None) -> tf.Tensor:
        causal_mask = self.get_causal_attention_mask(inputs)
        if mask is not None:
            padding_mask = tf.cast(mask[:, tf.newaxis, :], dtype="int32")
            padding_mask = tf.minimum(padding_mask, causal_mask)

        attention_output_1 = self.attention_1(
            query=inputs, value=inputs, key=inputs, attention_mask=causal_mask
        )
        attention_output_1 = self.dropout_1(attention_output_1)
        out_1 = self.layernorm_1(inputs + attention_output_1)

        attention_output_2 = self.attention_2(
            query=out_1,
            value=encoder_outputs,
            key=encoder_outputs,
            attention_mask=padding_mask,
        )
        attention_output_2 = self.dropout_2(attention_output_2)
        out_2 = self.layernorm_2(out_1 + attention_output_2)

        proj_output = self.dense_proj(out_2)
        proj_output = self.dropout_3(proj_output)
        return self.layernorm_3(out_2 + proj_output)

    def get_causal_attention_mask(self, inputs):
        input_shape = tf.shape(inputs)
        batch_size, sequence_length = input_shape[0], input_shape[1]
        i = tf.range(sequence_length)[:, tf.newaxis]
        j = tf.range(sequence_length)
        mask = tf.cast(i >= j, dtype="int32")
        mask = tf.reshape(mask, (1, input_shape[1], input_shape[1]))
        mult = tf.concat(
            [tf.expand_dims(batch_size, -1), tf.constant([1, 1], dtype=tf.int32)],
            axis=0,
        )
        return tf.tile(mask, mult)

The decoder transformer.

In [ ]:
class DecoderTransformer(layers.Layer):

    def __init__(self,
                 num_layers: int,
                 layers_size: int,
                 dense_size: int,
                 num_heads: int,
                 max_length: int,
                 v_size_dst: int,
                 dropout=0.1) -> None:
        super(DecoderTransformer, self).__init__()

        self.layers_size = layers_size
        self.num_layers = num_layers
        self.pos_embedding = PositionalEmbedding(max_length, v_size_dst, layers_size)
        self.dec_layers = [DecoderLayer(layers_size, dense_size, num_heads) for _ in range(num_layers)]
        self.dropout = layers.Dropout(dropout)
        self.supports_masking = True

    def call(self, inputs: tf.Tensor, enc_output: tf.Tensor, mask=None) -> tf.Tensor:
        dst_embeddings = self.pos_embedding(inputs)
        dec_output = self.dropout(dst_embeddings)
        for i in range(self.num_layers):
            dec_output = self.dec_layers[i](dec_output, enc_output)

        return dec_output

## **Building the model**

Next, we assemble the end-to-end model.

In [ ]:
with strategy.scope():
    encoder = encoder_model["encoder"]()

Downloading:   0%|          | 0.00/502M [00:00<?, ?B/s]

In [ ]:
def create_model(layers_size: int, num_layers: int, dense_size: int, num_heads: int, max_length: int, encoder=None) -> tf.keras.Model:
    # Encoder
    encoder_inputs = tf.keras.Input(shape=(None,), dtype="int32", name="encoder_inputs")
    if encoder is not None: 
        outputs = encoder(encoder_inputs)
        encoder_outputs = outputs.last_hidden_state
        layers_size = encoder_outputs.shape[-1]  # the size of the encoder and decoder layers must be the same
    else:
        encoder_outputs = EncoderTransformer(num_layers, layers_size, dense_size, num_heads, max_length, v_size_en)(encoder_inputs)

    # Decoder
    decoder_inputs = tf.keras.Input(shape=(None,), dtype="int32", name="decoder_inputs")
    encoded_seq_inputs = tf.keras.Input(shape=(None, layers_size), name="decoder_state_inputs")
    decoder_outputs = DecoderTransformer(num_layers, layers_size, dense_size, num_heads, max_length, v_size_it)(decoder_inputs, encoded_seq_inputs)
    decoder_outputs = layers.Dense(v_size_it, activation="softmax")(decoder_outputs)
    decoder = tf.keras.Model([decoder_inputs, encoded_seq_inputs], decoder_outputs, name="decoder_transformer")

    # Final model
    decoder_outputs = decoder([decoder_inputs, encoder_outputs])
    transformer = tf.keras.Model([encoder_inputs, decoder_inputs], decoder_outputs, name="transformer")
    return transformer

In [ ]:
with strategy.scope():
    opt = tf.keras.optimizers.Adam(learning_rate = 0.0001)
    transformer = create_model(512, 6, 2048, 8, max_length, encoder)
    transformer.summary()
    transformer.compile(opt, loss = "sparse_categorical_crossentropy", metrics=["accuracy"])

Model: "transformer"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 encoder_inputs (InputLayer)    [(None, None)]       0           []                               
                                                                                                  
 decoder_inputs (InputLayer)    [(None, None)]       0           []                               
                                                                                                  
 tf_bert_model (TFBertModel)    TFBaseModelOutputWi  108310272   ['encoder_inputs[0][0]']         
                                thPoolingAndCrossAt                                               
                                tentions(last_hidde                                               
                                n_state=(None, None                                     

## **Training the model**

We'll use accuracy as a quick way to monitor training progress on the validation data.
Note that machine translation typically uses BLEU scores as well as other metrics, rather than accuracy.

In [ ]:
with strategy.scope():
    model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(filepath='./translator_bert_euparl.h5', save_weights_only = True, monitor='val_loss', mode='auto', save_best_only=True)

In [ ]:
epochs = 10  # This should be at least 30 for convergence
transformer.fit(tr_batches, epochs=epochs, validation_data = val_batches, callbacks=[model_checkpoint_callback])

Epoch 1/10


INFO:absl:TPU has inputs with dynamic shapes: [<tf.Tensor 'Const:0' shape=() dtype=int32>, <tf.Tensor 'cond/Identity:0' shape=(None, 56) dtype=int32>, <tf.Tensor 'cond/Identity_8:0' shape=(None, 56) dtype=int32>, <tf.Tensor 'cond/Identity_16:0' shape=(None, 56) dtype=int32>]
INFO:absl:TPU has inputs with dynamic shapes: [<tf.Tensor 'Const:0' shape=() dtype=int32>, <tf.Tensor 'cond/Identity:0' shape=(None, 56) dtype=int32>, <tf.Tensor 'cond/Identity_8:0' shape=(None, 56) dtype=int32>, <tf.Tensor 'cond/Identity_16:0' shape=(None, 56) dtype=int32>]


5966/5966 [==============================] - ETA: 0s - loss: 1.5407 - accuracy: 0.5203

INFO:absl:TPU has inputs with dynamic shapes: [<tf.Tensor 'Const:0' shape=() dtype=int32>, <tf.Tensor 'cond/Identity:0' shape=(None, 56) dtype=int32>, <tf.Tensor 'cond/Identity_8:0' shape=(None, 56) dtype=int32>, <tf.Tensor 'cond/Identity_16:0' shape=(None, 56) dtype=int32>]


5966/5966 [==============================] - 2336s 367ms/step - loss: 1.5407 - accuracy: 0.5203 - val_loss: 0.9743 - val_accuracy: 0.6420
Epoch 2/10
5966/5966 [==============================] - 2165s 363ms/step - loss: 0.9540 - accuracy: 0.6441 - val_loss: 0.8219 - val_accuracy: 0.6792
Epoch 3/10
1338/5966 [=====>........................] - ETA: 27:08 - loss: 0.8784 - accuracy: 0.6623

In [ ]:
ts_loss, ts_accuracy = transformer.evaluate(make_batches(ts_set, batch_size))
print("Test loss: {0}\nTest accuracy: {1}".format(ts_loss, ts_accuracy))

NameError: ignored

In [ ]:
tf.keras.utils.plot_model(
    transformer, to_file='model_base.png', show_shapes=True, dpi=90
)

In [ ]:
transformer.save_weights('./translator_bert.h5', overwrite=True)

## **Translator**

In [ ]:
v_size_en = 30522

In [ ]:
tokenizer_it = BertTokenizer.from_pretrained(ita_src)
with strategy.scope():
  transformer = create_model(512, 6, 2048, 8, 11)
  transformer.load_weights('/content/drive/Shareddrives/HLT/translator_base.h5')

In [22]:
class Translator(tf.Module):

    def __init__(self, tokenizer_src, tokenizer_dst, max_length: int, transformer: tf.keras.Model) -> None:
        super(Translator, self).__init__()
        self.tokenizer_src = tokenizer_src
        self.tokenizer_dst = tokenizer_dst
        self.max_length = max_length
        self.transformer = transformer

    def translate(self, input_sentence: str) -> (list, str):
        tokenized_input_sentence = self.tokenizer_src(input_sentence, return_tensors='tf', add_special_tokens=True,
                                                max_length = self.max_length, padding='max_length', truncation=True).data["input_ids"]
        decoded_sentence = "[CLS]"
        list_tokens=[decoded_sentence]
        for i in range(self.max_length):
            decoded_sentence = self.tokenizer_dst.convert_tokens_to_string(list_tokens)
            tokenized_target_sentence = self.tokenizer_dst(decoded_sentence, return_tensors='tf', add_special_tokens=False,
                                                    max_length = self.max_length, padding='max_length', truncation=True).data['input_ids']
            predictions = self.transformer([tokenized_input_sentence, tokenized_target_sentence])
            sampled_token_index = np.argmax(predictions[0, i, :])
            sampled_token = self.tokenizer_dst.ids_to_tokens[sampled_token_index]
          
            if sampled_token == "[SEP]":
              decoded_sentence = self.tokenizer_dst.convert_tokens_to_string(list_tokens[1:])
              break
            
            list_tokens.append(sampled_token)
        
        return list_tokens, decoded_sentence

    def __translate_beam_search(self, input_sentence: str, k=2):
      tokenized_input_sentence = self.tokenizer_src(input_sentence, return_tensors='tf', add_special_tokens=True, max_length=self.max_length,
                        padding='max_length', truncation=True).data["input_ids"]
      decoded_sentence = (["[CLS]"], 1)
      beam_queue = SimpleQueue()
      beam_queue.put(decoded_sentence)
      translated = list()
      i = 0
      while not beam_queue.empty() and i < self.max_length:
          tokenized_sentence, prb = beam_queue.get()
          decoded_sentence = self.tokenizer_dst.convert_tokens_to_string(tokenized_sentence)
          tokenized_target_sentence = self.tokenizer_dst(decoded_sentence, return_tensors='tf', add_special_tokens=False, max_length=self.max_length,
                            padding='max_length').data['input_ids']
          predictions = self.transformer([tokenized_input_sentence, tokenized_target_sentence])
          i = len(tokenized_sentence) - 1
          sampled_token_indexes = np.argsort(predictions[0, i, :])[-k:]
          probabilities = [float(predictions[0, i, j]) for j in sampled_token_indexes]
          for samp_index, p in zip(sampled_token_indexes, probabilities):
              sampled_token = self.tokenizer_dst.ids_to_tokens[samp_index]
              tok_sent_with_new_samp = tokenized_sentence.copy()
              tok_sent_with_new_samp.append(sampled_token)
              next_sent = (tok_sent_with_new_samp, p * prb)
              if sampled_token == "[SEP]":  # and next_sent[1] > 0.02:
                  translated.append(next_sent)
              elif next_sent[1] > 0.001:
                  beam_queue.put(next_sent)

      translated.sort(key=lambda x: x[1], reverse=True)
      return translated

      def __call__(self, input_sentence: str, k=0):
          if k != 0:
              print("Niko")
              out_translation = self.translate(input_sentence)
          else:
              print("Niko, beam search")
              out_translation = self.__translate_beam_search(input_sentence, k)
          
          return out_translation

In [ ]:
with strategy.scope():
    translator = Translator(encoder_model["tokenizer_translation"], BertTokenizer.from_pretrained(ita_src), 11, transformer)

en = "I saw a bed in your bedroom."
out_translation = translator.translate(en)
print(out_translation[-1])

I primi bombardieri americani hanno eseguito le armi da


## **Bleu score**

In [23]:
import tarfile
fname = "/content/drive/Shareddrives/HLT/evaluation/flores101_dataset.tar.gz"
if fname.endswith("tar.gz"):
    tar = tarfile.open(fname, "r:gz")
    tar.extractall()
    tar.close()
elif fname.endswith("tar"):
    tar = tarfile.open(fname, "r:")
    tar.extractall()
    tar.close()

In [24]:
with open("/content/flores101_dataset/devtest/eng.devtest", encoding="UTF-8") as datafile:
    src_test = list()
    for sentence in datafile:
        src_test.append(sentence)

In [25]:
max_length_bleu = set_max_tokens(src_test, "en")

NameError: ignored

In [ ]:
tokenizer_it = BertTokenizer.from_pretrained(ita_src)
with strategy.scope():
    transformer = create_model(512, 6, 2048, 8, 24, encoder)
    transformer.load_weights('drive/Shareddrives/HLT/translator_bert.h5')
    translator = Translator(encoder_model["tokenizer_translation"], BertTokenizer.from_pretrained(ita_src), max_length, transformer)

In [ ]:
dst_out = list()
for sentence in tqdm_notebook(src_test):
    dst_out.append(translator.translate(sentence)[-1])

  0%|          | 0/1012 [00:00<?, ?it/s]

KeyboardInterrupt: ignored

In [ ]:
with open("detok_trans.txt", encoding="UTF-8") as datafile:
    datafile.write(dst_out)

In [36]:
!git clone --single-branch --branch adding_spm_tokenized_bleu https://github.com/ngoyal2707/sacrebleu.git
%cd sacrebleu
!python setup.py install
%cd ../

Cloning into 'sacrebleu'...
remote: Enumerating objects: 1087, done.
remote: Counting objects: 100% (6/6), done.
remote: Compressing objects: 100% (6/6), done.
remote: Total 1087 (delta 0), reused 5 (delta 0), pack-reused 1081
Receiving objects: 100% (1087/1087), 378.97 KiB | 4.03 MiB/s, done.
Resolving deltas: 100% (715/715), done.
/content/sacrebleu
/usr/local/lib/python3.7/dist-packages/setuptools/dist.py:700: UserWarning: Usage of dash-separated 'description-file' will not be supported in future versions. Please use the underscore name 'description_file' instead
  % (opt, underscore_opt))
running install
running bdist_egg
running egg_info
creating sacrebleu.egg-info
writing sacrebleu.egg-info/PKG-INFO
writing dependency_links to sacrebleu.egg-info/dependency_links.txt
writing entry points to sacrebleu.egg-info/entry_points.txt
writing requirements to sacrebleu.egg-info/requires.txt
writing top-level names to sacrebleu.egg-info/top_level.txt
writing manifest file 'sacrebleu.egg-info

In [50]:
!cat drive/Shareddrives/HLT/evaluation/detok_trans.txt | sacrebleu -tok spm flores101_dataset/devtest/ita.devtest --force --score-only 

sacreBLEU: That's 100 lines that end in a tokenized period ('.')
sacreBLEU: It looks like you forgot to detokenize your test data, which may hurt your score.
sacreBLEU: If you insist your data is detokenized, or don't care, you can suppress this message with '--force'.
0.8
